# Imports

In [7]:
import sys
device: str
if 'google.colab' in sys.modules.keys():
    device = 'colab'
if 'kaggle_web_client' in sys.modules.keys():
    device = 'kaggle'
else:
    device = 'locally'

In [9]:
def install_libaries():
    if device == 'kaggle':
        !conda create -y -q -d --name create_models_venv numpy tensorflow=2.9.1 pandas seaborn
        # create a conda virtual env with the liberies: numpy tensorflow pandas matplotlib seaborn
        # -y is for dont ask if Im sure
        !pip install -q tensorflow-text  # some might not be avilable using conda install so pip install them
        !pip install -q tqdm
    elif device == 'colab':
        %pip install -q tensorflow-text
        %pip install -q tqdm
        %pip install -q wandb --upgrade
        # q for quiet

In [10]:
# standart liberies:
from typing import Optional, List, Set, Dict, Tuple
import datetime
import os
import random
import statistics
import math
import time
import copy
# NON-standart liberies:
# from flopco_keras import FlopCoKeras  # flop counter for keras at https://github.com/evgps/flopco-keras
try:
    import wandb
    import tensorflow as tf
    from tensorflow import keras
    from tensorflow.keras import layers, Model
    import pandas as pd
    import seaborn as sns
    import numpy as np
    import tqdm
    from tensorflow_text.tools.wordpiece_vocab import bert_vocab_from_dataset
    from tensorflow_text import FastWordpieceTokenizer
except ModuleNotFoundError:
    install_libaries()
    from tensorflow_text.tools.wordpiece_vocab import bert_vocab_from_dataset
    from tensorflow_text import FastWordpieceTokenizer
except ImportError:
    install_libaries()
    from tensorflow_text.tools.wordpiece_vocab import bert_vocab_from_dataset
    from tensorflow_text import FastWordpieceTokenizer


CondaValueError: Cannot `create --dry-run` with an existing conda environment

^C
ERROR: Operation cancelled by user


KeyError: 'tensorflow'

In [ ]:
from tensorflow_text.tools.wordpiece_vocab import bert_vocab_from_dataset
from tensorflow_text import FastWordpieceTokenizer
import pandas as pd
import seaborn as sns
import numpy as np
import tqdm

In [ ]:
print(f"Python version: {sys.version}")
print(f"Tensorflow version: {tf.__version__}")

# Settings

In [ ]:
tf.random.set_seed(0)
random.seed(0)
np.random.seed(0)
keras.utils.set_random_seed(0)
tf.config.experimental.enable_op_determinism()
# keras.backend.set_floatx('float16')
f_type = keras.backend.floatx()  # either tf.float16 or tf.float32

## Define a strategy - Accelerator optimization 

In [ ]:
try:
    resolver = tf.distribute.cluster_resolver.TPUClusterResolver();
    tf.config.experimental_connect_to_cluster(resolver);
    tf.tpu.experimental.initialize_tpu_system(resolver);
    strategy = tf.distribute.TPUStrategy(resolver);
except ValueError:
    pass

# Data loading

In [ ]:
if device == 'colab':  # If notebook is ran on colab
    from google.colab import drive
    drive.mount('/drive')
    df: pd.DataFrame = pd.read_csv('/drive/MyDrive/final_project/wikipedia_articles.csv')
elif device == 'kaggle':
    df: pd.DataFrame = pd.read_csv('../input/wikipedia-promotional-articles/promotional.csv')
else:  # If notebook is ran on my laptop
    df: pd.DataFrame = pd.read_csv('wiki_data/articles.csv')
print(f"the shape of the dataframe: {df.shape}")

In [ ]:
df: pd.Series = df['text']
data_list: List[str] = df.to_list()
DATA_SIZE = len(data_list)
print(f"There are {DATA_SIZE} data points")
string_lengths: List[int] = [len(data_point) for data_point in data_list]
max_string_len = max(string_lengths)
print(f"The length of the longest text IN CHARACTERS is: {max_string_len}")
min_string_len = min(string_lengths)
print(f"The length of the shortest text IN CHARACTERS is: {min_string_len}")

## Creating the vocabulary and tokenizer

In [ ]:
# %%time
bert_tokenizer_params: dict = dict(lower_case=True)
VOCAB_SIZE: int = 8192  # Always the same for all models

if device == 'colab':  # If notebook is ran on colab
    path = '/drive/MyDrive/final_project/vocab.txt'
if device == 'kaggle':
    path = "../input/my-lookup-table/look_up_table.txt"
else:  # If notebook is ran on my laptop
    path = 'C:/yoni/final_project/model/vocab.txt'


reserved_tokens: List[str] = ["[PAD]", "[UNK]", "[START]", "[END]", "[MASK]"]

if os.path.exists(path):
    with open(path, 'r') as f:
        vocab: List[str] = f.read().split()
else:
    bert_vocab_args: dict = dict(
        # The target vocabulary size
        vocab_size = VOCAB_SIZE,
        # Reserved tokens that must be included in the vocabulary
        reserved_tokens=reserved_tokens,
        # Arguments for `tf_text.BertTokenizer`
        bert_tokenizer_params=bert_tokenizer_params,
        # Arguments for `wordpiece_vocab.wordpiece_tokenizer_learner_lib.learn`
        learn_params={},
    )
    tensor_list: list = [tf.convert_to_tensor(data_point) for data_point in data_list]
    data_set: tf.data.Dataset = tf.data.Dataset.from_tensor_slices(tensor_list)
    # I already ran this code and saved the file to C:/yoni/final_project/model/vocab.txt
    vocab: List[str] = bert_vocab_from_dataset(
        data_set,
        **bert_vocab_args,)
    with open('C:/yoni/final_project/model/vocab.txt', 'w') as f:
        for token in vocab:
            f.write(token + ' ')
print(f"the type of the items in vocab: {type(vocab[0])}")
print(f"the first 15 items in vocab: {vocab[:15]}")
vocab_size = len(vocab)
print(f" the length of vocab: {vocab_size}")
tokenizer = FastWordpieceTokenizer(vocab, support_detokenization=True, token_out_type=tf.int32)

In [ ]:
x = tf.Ragged([["this is my string"]])
print(x)
print(tokenizer.tokenize(x))

## Tokenizing the data

In [ ]:
# START: int = tf.argmax(tf.constant(reserved_tokens) == "[START]")  # The value of the start token
# END: int = tf.argmax(tf.constant(reserved_tokens) == "[END]")  # The value of the end token
# starts = tf.cast(tf.Variable([START]), dtype = tf.int32)  # Tensor of shape [1] and dtype int
# ends = tf.cast(tf.Variable([END]), dtype = tf.int32)  # Tensor of shape [1] and dtype int
starts = tf.constant([2], dtype=tf.int32)
ends = tf.constant([3], dtype=tf.int32)
pad_int: int = int(tf.argmax(tf.constant(reserved_tokens) == "[PAD]"))
pad_ten: tf.TensorSpec(dtype=tf.int32, shape=()) = tf.constant([pad_int], dtype=tf.int32)

In [ ]:
def tokenize_string(text: str) -> tf.Tensor:
    """Converts string to tensor"""
    ragged: tf.RaggedTensor = tokenizer.tokenize(text)
    eager: tf.Tensor = ragged.to_tensor(default_value=0, shape=[None, 1])  # 0 is the value of the padding token
    sqeezed: tf.Tensor = tf.squeeze(eager, axis=1)
    typed: tf.Tensor = tf.cast(sqeezed, tf.int32)
    return edited

In [ ]:
tokenized_data: List[tf.Tensor] = [tokenize_string(data_point) for data_point in data_list] 
print(f"An example of a shape of tokens tensor: {tokenized_data[0].shape}")
print(f"examples of tokens tensor: {tokenized_data[0][:10]}")
lengths_tokenized: List[int] = [text.shape[0] for text in tokenized_data]
print(f"The maximum length of a tokens tensor: {max(lengths_tokenized)}")
print(f"The minimum length of a tokens tensor: {min(lengths_tokenized)}")
sns.displot(lengths_tokenized);

### chunk too long texts

In [ ]:
max_seq_len: int = 256
def chunk_tensor(tensor: tf.Tensor, max_len: int = max_seq_len) -> List[tf.Tensor]:
    """Splits 1d tensor to chunks (1d tensors) of maximum size: max_len"""
    return [tensor[i*max_len:(i+1)*max_len] for i in range(tensor.shape[0] // max_len)]

In [ ]:
chunked_data: List[tf.Tensor] = []
for tensor in tokenized_data:
    chunks = chunk_tensor(tensor, max_seq_len)
    for chunk in chunks:
        chunked_data.append(chunk)
DATA_SIZE: int = len(chunked_data)
print(f"The data set size after chunking {DATA_SIZE}")
print(f"An example of a chunked shape{chunked_data[0].shape}")

## Padding

In [ ]:
def pad(tensor: tf.Tensor, pad_int: int) -> tf.Tensor:
    """Pads the tensor to the length of the longest text in the data set"""
    padded: tf.Tensor = tf.pad(tensor=tensor, paddings=[[pad_int, max_seq_len - tensor.shape[0]]], mode='CONSTANT', constant_values=0)
    # 0 is the padding token
    return padded

In [ ]:
padded_data: List[tf.Tensor] = [pad(text, pad_int) for text in chunked_data]
chunked_data.sort(key = lambda t: t.shape[0])  # sorting so that every batch will have similar sized texts, used when training

## Train test val split

In [ ]:
batch_size: int = 128

def list_to_dataset(tokenized_list: List[tf.Tensor]) -> tf.data.Dataset:
    """Converts a list of tokenized texts after all preprocessing to a tf.data.Dataset"""
    dataset: tf.data.Dataset = tf.data.Dataset.from_tensor_slices(tokenized_list)
    batched: tf.data.Dataset = dataset.batch(batch_size)
    return batched

batched_data_ten = list_to_dataset(padded_data)
batched_data_list = list(batched_data_ten)
random.shuffle(batched_data_list)
if batched_data_list[-1].shape[0] != batch_size:
    batched_data_list = batched_data_list[:-1]
data_size = len(batched_data_list)
train_size: int = int(data_size * 0.8) 
val_test_size: int = int(data_size * 0.1)  # Both validation and test get 10% of the data
list_train_set: List[tf.Tensor] = batched_data_list[:train_size]
list_val_set: List[tf.Tensor] = batched_data_list[train_size:(train_size + val_test_size)]
list_test_set: List[tf.Tensor] = batched_data_list[(train_size + val_test_size)]

## Clear memory

In [ ]:
del batched_data_list, train_size, val_test_size, data_size
del padded_data, chunked_data, tokenized_data, data_list, df, lengths_tokenized, reserved_tokens
del bert_tokenizer_params, ends, starts, vocab
del chunk, chunks

# Model


## Positional encoding

The formula for calculating the positional encoding is as follows:

$${PE_{(pos, 2i)} = \sin(pos / 10000^{2i / d_{model}})} $$
$${PE_{(pos, 2i+1)} = \cos(pos / 10000^{2i / d_{model}})} $$

where $d_{model}$ is the model dimension, $pos$ is the position and $i$ is the index of the embedding.
this is taken from the paper: attention is all you need.

In [ ]:
def create_positional_encoding(max_len: int, d_model: int) -> tf.Tensor:
    """Returns the positional encoding for a given a maximal sequence length and model dimension.
    used in SeTransformer.__init__()
    inputs: max_len: int, d_model: int
    returns: tf.Tensor of shape (1, max_len, d_model) and dtype f_type
    The 1 is for the batch dimension, the place in the batch dimension does not matter"""

    def get_angles(positions: np.ndarray, timestamps: np.ndarray, d_model: int) -> np.ndarray:
        """Returns the angle in radians for given positions, timestamps and the dimension of the model
        input: positions: np.ndarray of shape (max_len, 1), timestamps: np.ndarray of shape (1, d_model), d_model: int
        output: np.ndarray of shape (max_len, d_model)"""
        if f_type == "float32":
            angle_rates = 1 / np.power(10000, ((2 * (timestamps//2)) / np.float32(d_model)))
        else:
            angle_rates = 1 / np.power(10000, ((2 * (timestamps//2)) / np.float16(d_model)))

        return positions * angle_rates
    
    angle_rads = get_angles(np.arange(max_len)[:, np.newaxis],
                            np.arange(d_model)[np.newaxis, :],
                            d_model)  # (max_len, d_model)

    # apply sin to even indices in the array; 2i for i in range(d_model // 2)
    angle_rads[:, 0::2] = np.sin(angle_rads[:, 0::2])  # (max_len, d_model)

    # apply cos to odd indices in the array; 2i+1
    angle_rads[:, 1::2] = np.cos(angle_rads[:, 1::2])  # (max_len, d_model)

    pos_encode = angle_rads[np.newaxis, ...]  # (1, max_len, d_model)

    return tf.cast(pos_encode, dtype=f_type)

## Masking

Mask all the pad tokens in the batch of sequence. It ensures that the model does not treat padding as the input. The mask indicates where pad value 0 is present: it outputs a 1 at those locations, and a 0 otherwise.

The look-ahead mask is used to mask the future tokens in a sequence. In other words, the mask indicates which entries should not be used.

This means that to predict the third token, only the first and second token will be used. Similarly to predict the fourth token, only the first, second and the third tokens will be used and so on.

In [ ]:
def create_masks(inp: tf.Tensor, tar: tf.Tensor, pad_ten: tf.Tensor) -> Tuple[tf.Tensor, tf.Tensor]:
        """Creates all the masks needed for the model
        input: inp: tf.Tensor of shape (batch_size, seq_len), tar: tf.Tensor of shape (batch_size, set_size)
        Returns: tuple of (padding_mask, look_ahead_mask)
        padding_mask, look_ahead_mask: tf.Tensor of shape (batch_size, 1, 1, seq_len)"""
        
        def create_padding_mask(seq: tf.Tensor) -> tf.Tensor:
                """Returns a padding mask for the given sequence.
                input: seq: tf.Tensor of shape (batch_size, seq_len)
                Returns: tf.Tensor of shape (batch_size, 1, 1, seq_len)"""
                seq = tf.cast(tf.math.equal(seq, pad_ten), f_type)  
                # For every item in the sequence, 1 if it is a padding token, 0 if it is not 

                # add extra dimensions to add the padding
                
                return seq[:, tf.newaxis, tf.newaxis, :]  # (batch_size, 1, 1, seq_len)
        
        # Encoder padding mask
        padding_mask: tf.Tensor = create_padding_mask(inp)  # (batch_size, 1, 1, seq_len)

        # Used in the 1st attention block in the decoder.
        # It is used to pad and mask future tokens in the input received by
        # the decoder.
        set_size: int = tar.shape[1]

        def create_look_ahead_mask(set_size: int) -> tf.Tensor:
                mask = 1 - tf.linalg.band_part(tf.ones((set_size, set_size)), -1, 0)
                mask = tf.cast(mask, dtype=f_type)
                return mask  # (seq_len, seq_len)

        look_ahead_mask = create_look_ahead_mask(set_size)  # (seq_len, seq_len)
        dec_target_padding_mask = create_padding_mask(tar)  # (batch_size, 1, 1, seq_len)
        look_ahead_mask = tf.maximum(dec_target_padding_mask, look_ahead_mask) # (batch_size, 1, 1, seq_len)

        return padding_mask, look_ahead_mask

## Layers and blocks

In [ ]:
class ScaledDotProductAttention(layers.Layer):
    def __init__(self, d_model: int, **kwargs):
        super(ScaledDotProductAttention, self).__init__(**kwargs)
        # scale = 1 / sqrt(d_model)
        self.scale = tf.math.pow(tf.cast(d_model, f_type), -0.5)
        self.softmax = layers.Softmax(axis=-1)

    def call(self, q: tf.Tensor, k: tf.Tensor, v: tf.Tensor, mask: Optional[tf.Tensor] = None) -> tf.Tensor:
        """Scaled Dot-Product Attention
        input: 
        q: tf.Tensor of shape (batch_size, seq_len, d_model), 
        k: tf.Tensor of shape (batch_size, seq_len, d_model), 
        v: tf.Tensor of shape (batch_size, seq_len, d_model), 
        mask: Optional[tf.Tensor] of shape (batch_size, 1, 1, seq_len)
        output: tf.Tensor of shape (batch_size, seq_len, d_model)"""
        matmul_qk: tf.Tensor = tf.matmul(q, k, transpose_b=True)  # (..., seq_len_q, seq_len_k)
        # q @ transpose(k)

        # Scaled Dot-Product Attention
        scaled_attention_logits: tf.Tensor = matmul_qk * self.scale  # (..., seq_len_q, seq_len_k)
        # matmul_qk / sqrt(d_model)

        # Masking
        if mask is not None:
            # noinspection PyTypeChecker
            if f_type == 'float16':
                # tf.float16.min is minus infinity
                scaled_attention_logits += (mask * tf.float16.min)  # changed from -1e9 to prevent nan's
            else:
                scaled_attention_logits += (mask * -1e9) 

        # Normalize
        attention_weights = self.softmax(scaled_attention_logits)
        # (..., seq_len_q, seq_len_k)

        # Output
        output = tf.matmul(attention_weights, v)

        return output

In [ ]:
class MyMultiHeadAttention(keras.Model):
    """U can use the built-in layers.multihead_attention but is caused a bug for me"""
    def __init__(self, num_heads: int, d_model: int, **kwargs):
        super(MyMultiHeadAttention, self).__init__(**kwargs)
        if d_model % num_heads != 0:
            raise ValueError(f"d_model ({d_model}) must be divisible by num_heads ({num_heads})")
        self.num_heads = num_heads
        self.d_model = d_model
        self.depth = d_model // self.num_heads

        self.wq = layers.Dense(d_model)
        self.wk = layers.Dense(d_model)
        self.wv = layers.Dense(d_model)

        self.dense = layers.Dense(d_model)
        self.sdpa = ScaledDotProductAttention(d_model)

        
    def split_heads(self, x: tf.Tensor, batch_size: int) -> tf.Tensor:
        """Split the last dimension into (num_heads, depth).
        Transpose the result such that the shape is (batch_size, num_heads, seq_len, depth)
        """
        x = tf.reshape(x, (batch_size, -1, self.num_heads, self.depth))
        return tf.transpose(x, perm=[0, 2, 1, 3])

    
    def call(self, v_k: tf.Tensor, q: tf.Tensor, mask: tf.Tensor) -> tf.Tensor:
        """inputs:
        v_k: tf.Tensor of shape (batch_size, seq_len, d_model) in self attention keys and values are the same
        q: tf.Tensor of shape (batch_size, seq_len, d_model)
        mask: Optional[tf.Tensor] of shape (batch_size, seq_len)"""
        batch_size = tf.shape(q)[0]

        q: tf.Tensor = self.wq(q)  # (batch_size, seq_len, d_model)
        k: tf.Tensor = self.wk(v_k)  # (batch_size, seq_len, d_model)
        v: tf.Tensor = self.wv(v_k)  # (batch_size, seq_len, d_model)

        q: tf.Tensor = self.split_heads(q, batch_size)  # (batch_size, num_heads, seq_len_q, depth)
        k: tf.Tensor = self.split_heads(k, batch_size)  # (batch_size, num_heads, seq_len_k, depth)
        v: tf.Tensor = self.split_heads(v, batch_size)  # (batch_size, num_heads, seq_len_v, depth)

        # scaled_attention.shape should be (batch_size, num_heads, seq_len_q, depth)
        scaled_attention = self.sdpa(q, k, v, mask)

        scaled_attention = tf.transpose(scaled_attention, perm=[0, 2, 1, 3]) 
         # (batch_size, seq_len_q, num_heads, depth)

        concat_attention = tf.reshape(scaled_attention, (batch_size, -1, self.d_model))
          # (batch_size, seq_len_q, d_model)

        output = self.dense(concat_attention)  # (batch_size, seq_len_q, d_model)

        return output

In [ ]:
class PointWiseFeedForward(keras.Model):
    def __init__(self, d_model: int, dff: int, **kwargs): 
        super(PointWiseFeedForward, self).__init__(**kwargs)
        self.layer1 = layers.Dense(dff, activation='relu')  # (batch_size, seq_len, dff)
        self.layer2 = layers.Dense(d_model)  # (batch_size, seq_len, d_model)
    
    def call(self, x: tf.Tensor) -> tf.Tensor:
        """Gets and returns tensor of shape (batch_size, seq_len, d_model) and dtype keras.beckend.floatx()"""
        x = self.layer1(x)
        x = self.layer2(x)
        return x

In [ ]:
class EncoderBlock(keras.Model):
    def __init__(self, d_model: int, num_heads: int, dff: int, drop_out_rate: float, **kwargs):
        super(EncoderBlock, self).__init__(**kwargs)

        self.mha = MyMultiHeadAttention(num_heads = num_heads, d_model = d_model)
        self.ffn = PointWiseFeedForward(d_model, dff)

        self.layer_norm = layers.LayerNormalization(epsilon=1e-6)

        self.dropout = layers.Dropout(drop_out_rate)

    def call(self, x: tf.Tensor, training: bool, mask: tf.Tensor) -> tf.Tensor:
        
        attn_output = self.mha(x, x, mask)  # (batch_size, input_seq_len, d_model)
        attn_output = self.dropout(attn_output, training=training)  # (batch_size, input_seq_len, d_model)
        # out1 = self.layer_norm(x + attn_output)  # (batch_size, input_seq_len, d_model)
        # might be data leak
        out1 = self.layer_norm(attn_output)  # (batch_size, input_seq_len, d_model)
        
        ffn_output = self.ffn(out1)  # (batch_size, input_seq_len, d_model)
        ffn_output = self.dropout(ffn_output, training=training)  # (batch_size, input_seq_len, d_model)
        out2 = self.layer_norm(out1 + ffn_output)  # (batch_size, input_seq_len, d_model)

        return out2

In [ ]:
class DecoderBlock(keras.Model):
    def __init__(self, d_model: int, num_heads: int, dff: int, rate: float, **kwargs):
        super(DecoderBlock, self).__init__(**kwargs)

        self.mha = MyMultiHeadAttention(num_heads = num_heads, d_model = d_model)

        self.ffn = PointWiseFeedForward(d_model, dff)

        self.layer_norm = layers.LayerNormalization(epsilon=1e-6)

        self.dropout = layers.Dropout(rate)

    def call(self, x: tf.Tensor, enc_output: tf.Tensor, look_ahead_mask: tf.Tensor, padding_mask: tf.Tensor, training):
        # enc_output.shape should be (batch_size, input_seq_len, d_model)

        attn1 = self.mha(x, x, look_ahead_mask)  # (batch_size, set_size, d_model)
        attn1 = self.dropout(attn1, training=training)  # (batch_size, set_size, d_model)
        # out1 = self.layer_norm(attn1 + x)
        # might be data leak
        out1 = self.layer_norm(attn1)  # (batch_size, set_size, d_model)

        attn2 = self.mha(enc_output, out1, padding_mask)  # (batch_size, set_size, d_model)
        attn2 = self.dropout(attn2, training=training)  # (batch_size, set_size, d_model)
        out2 = self.layer_norm(attn2 + out1)  # (batch_size, set_size, d_model)

        ffn_output = self.ffn(out2)  # (batch_size, set_size, d_model)
        ffn_output = self.dropout(ffn_output, training=training)
        out3 = self.layer_norm(ffn_output + out2)  # (batch_size, set_size, d_model)

        return out3 

In [ ]:
class Encoder(keras.Model):
    def __init__(self, pos_encoding: tf.Tensor, num_blocks: int, d_model: int, num_heads: int, dff: int, rate=0.1, **kwargs):
        super(Encoder, self).__init__(**kwargs)

        self.d_model = d_model
        self.num_blocks = num_blocks
        self.pos_encoding = pos_encoding

        self.enc_blocks = [EncoderBlock(d_model, num_heads, dff, rate) for _ in range(num_blocks)]
        # the encoder 
        self.dropout = layers.Dropout(rate)
        self.scale = tf.math.sqrt(tf.cast(self.d_model, f_type))

    def call(self, x: tf.Tensor, training, mask: tf.Tensor) -> tf.Tensor:

        seq_len = tf.shape(x)[1]

        # adding position encoding.
        # assert not tf.math.is_nan(x[0][0][0])
        x *= self.scale
        # assert not tf.math.is_nan(x[0][0][0])
        
        x += self.pos_encoding[:, :seq_len, :]  # (batch_size, input_seq_len, d_model)
        # assert not tf.math.is_nan(x[0][0][0])
        x = self.dropout(x, training=training)  # (batch_size, input_seq_len, d_model)
        # assert not tf.math.is_nan(x[0][0][0])

        for block in self.enc_blocks:
            x = block(x, training, mask)  # (batch_size, input_seq_len, d_model)
            # assert not tf.math.is_nan(x[0][0][0])

        return x  # (batch_size, input_seq_len, d_model)  

In [ ]:
class Decoder(keras.Model):
    def __init__(self, pos_encoding, num_blocks: int, d_model: int, num_heads: int, dff: int,
                 vocab_size: int, rate: float, **kwargs):
        super(Decoder, self).__init__(**kwargs)

        self.scale = tf.math.sqrt(tf.cast(d_model, f_type))
        self.num_blocks = num_blocks
        self.pos_encoding = pos_encoding

        self.embedding = layers.Embedding(vocab_size, d_model)
        self.dec_blocks = [DecoderBlock(d_model, num_heads, dff, rate) for _ in range(num_blocks)]
        self.dropout = layers.Dropout(rate)

    def call(self, tar: tf.Tensor, enc_output: tf.Tensor, training: bool,
             look_ahead_mask: tf.Tensor, padding_mask: tf.Tensor) -> tf.Tensor:

        seq_len = tf.shape(tar)[1]

        x = self.embedding(tar)  # (batch_size, set_size, d_model)
        x *= self.scale
        x += self.pos_encoding[:, :seq_len, :]

        x = self.dropout(x, training=training)

        for block in self.dec_blocks:
            x = block(x=x, enc_output=enc_output, look_ahead_mask=look_ahead_mask,
                      padding_mask=padding_mask, training=training)

        # x.shape should be (batch_size, set_size, d_model)
        return x

In [ ]:
class EmbeddingTransposed(layers.Layer):
    def __init__(self, tied_to: layers.Embedding = None, activation: Optional[str] = None, **kwargs):
        super(EmbeddingTransposed, self).__init__(**kwargs)
        self.tied_to = tied_to
        self.activation = keras.activations.get(activation)

    def build(self, input_shape):
        self.custom_weights = self.tied_to.weights[0]
        self.built = True

    def compute_output_shape(self, input_shape):
        return input_shape[0], keras.backend.int_shape(self.tied_to.weights[0])[0]

    def call(self, inputs, mask=None):
        output = keras.backend.dot(inputs, keras.backend.transpose(self.custom_weights))
        if self.activation is not None:
            output = self.activation(output)
        return output

    def get_config(self):
        config = {"activation": keras.activations.serialize(self.activation)}
        base_config = super(EmbeddingTransposed, self).get_config()
        return dict(list(base_config.items()) + list(config.items()))

## The full model

In [ ]:
def count_layers(my_model) -> int:
    """Counts the layers of a keras model recursizely"""
    if not isinstance(my_model, keras.Model): 
        if isinstance(my_model, layers.Layer):
            return 1
        return 0
    return sum([count_layers(sub_model) for sub_model in my_model.layers])

In [ ]:
class SeTransformer(keras.Model):
    """The base architecture of my models in this project."""
    def __init__(self, num_blocks: int, d_model: int, num_heads: int, dff: int,
                 vocab_size: int, max_len: int, rate: float, pad_int: int, **kwargs):
        super(SeTransformer, self).__init__(**kwargs)  # calls keras.Model's __init__ method with kwarg as key worg arguments
        self.pad_int = pad_int
        self.vocab_size = vocab_size
        pos_encoding = create_positional_encoding(max_len, d_model)
        self.d_model = d_model
        self.encoder = Encoder(pos_encoding, num_blocks, d_model, num_heads, dff, rate)
        self.decoder = Decoder(pos_encoding, num_blocks, d_model, num_heads, dff, vocab_size, rate)
        self.embedding = layers.Embedding(input_dim=vocab_size, output_dim=d_model)
        self.emb_trans = EmbeddingTransposed(self.embedding, "softmax")


    def get_layer_count(self) -> int:
        return count_layers(self)
    
    
    def summary(self, **kwargs) -> None:
        super().summary(**kwargs)
        print(f"The model have {self.get_layer_count()} layers")
        
    
    def count_params(self) -> int:
        """counts trainable parameters
        Raises an error if caleed before building the model"""
        param_count: int = self.encoder.count_params() + self.decoder.count_params() + self.embedding.count_params()
        return param_count
    
    
    def build_graph(self) -> keras.Model:
        """Returns a functional keras model identical to the model"""
        inp = layers.Input(shape=(batch_size, max_seq_len))
        tar = layers.Input(shape=(batch_size, set_size))
        return keras.Model(inputs=[[inp, tar], True], outputs=self.call([inp, tar], True))
    
    
    def call(self, inputs: List[tf.Tensor], training: bool) -> tf.Tensor:
        inp, tar = inputs
        # inp.shape should be (batch_size, max_seq_len)
        # tar.shape should be (batch_size, set_size)
        x = self.embedding(inp)  # (batch_size, max_seq_len, d_model)
        padding_mask, look_ahead_mask = create_masks(inp, tar, self.pad_int)
        enc_output = self.encoder(x, training, padding_mask)  # (batch_size, max_seq_len, d_model)
        dec_output = self.decoder(tar, enc_output, training, look_ahead_mask, padding_mask)  # (batch_size, set_size, d_model)
        final_output = self.emb_trans(dec_output)  # (batch_size, set_size, vocab_size)
        return final_output

In [ ]:
sample_transformer = SeTransformer(
    num_blocks=2, d_model=32, num_heads=4, dff=128,
    vocab_size=7000,
    max_len=1200, pad_int=0, rate=0.1)

temp_input = tf.random.uniform((1, 200), dtype=tf.int32, minval=5, maxval=6999)
temp_target = tf.random.uniform((1, 8), dtype=tf.int32, minval=5, maxval=6999)
temp_target2 = temp_target + 3

train_out = sample_transformer([temp_input, temp_target], training=True)
train_out2 = sample_transformer([temp_input, temp_target2], training=True)

try:
    tf.debugging.assert_equal(train_out, train_out2)
except tf.errors.InvalidArgumentError:
    print("WARNING: model output might depends on the target")

del sample_transformer, train_out, temp_target2, train_out2

# Training the model

In [ ]:
# model_art = wandb.use_artifact(f"{model_collection_name}:latest")
# model_path = model_art.get_path("model.pb").download()
# model = tf.saved_model.load(model_path)

## Hyper-Parameters

In [ ]:
set_size: int = 2
learning_rate: float = 0.005

num_sets: int = (max_seq_len // set_size) - 1 # Because we dont predict the first set
# number of sets in each sequence

num_blocks: int = 16
d_model: int = 512
dff: int = 1024
num_heads: int = 32
dropout_rate: float = 0.1

## Weights and Biases

In [ ]:
if device == 'kaggle':
    try:
        from kaggle_secrets import UserSecretsClient
        user_secrets = UserSecretsClient()
        os.environ['WANDB_API_KEY'] = user_secrets.get_secret("WANDB_API_KEY")
    except Exception:
        print("please enter your weights and biases API key")
!wandb login

In [ ]:
# try: 
#     artifect = use_artifact(artifact, use_as=None)
#     art = wandb.use_artifact(...)
#     wandb.run.link_artifact(art, 'yonikremer/final_project_owned/version0')

In [ ]:
if not 'run' in globals():
    run = wandb.init(
        project="final_project_owned",
        entity="yonikremer",
        name=datetime.datetime.today().strftime(f"run from %d/%m/%Y"),
        settings=wandb.Settings(start_method="thread"),
        config = {"set size": set_size,
                  "batch size": batch_size,
                  "learning rate": learning_rate,
                  "max seq len": max_seq_len,
                  "num blocks": num_blocks,
                  "model dimention": d_model,
                  "dff": dff,
                  "num heads": num_heads,
                  "dropout rate": dropout_rate
                  })
    config = wandb.config

## Create the model

In [ ]:
model = SeTransformer(
    num_blocks=num_blocks,
    d_model=d_model,
    num_heads=num_heads,
    dff=dff,
    vocab_size=vocab_size,
    max_len=max_seq_len,
    rate=dropout_rate,
    pad_int=pad_int)

optimizer = keras.optimizers.Adam(learning_rate, epsilon=keras.backend.epsilon())
loss_func = keras.losses.SparseCategoricalCrossentropy(from_logits=False)
acc = keras.metrics.SparseCategoricalAccuracy(dtype=f_type)

model.compile(optimizer=optimizer,
    loss=loss_func,
    metrics=[acc]
 )

In [ ]:
temp_input = tf.random.uniform((batch_size, max_seq_len), dtype=tf.int32, minval=5, maxval=6999)
temp_target = tf.random.uniform((batch_size, set_size), dtype=tf.int32, minval=5, maxval=6999)
temp_pred = model([temp_input, temp_target], False)

param_count: int = model.count_params()
print(f"The model has {param_count:,} = {round(param_count * (10**-6), 1)}M trainable parameters")
run.config["parameters"] = param_count

# stats = FlopCoKeras(model)
# flops_per_call: int = stats.total_flops
# macs_per_call: int = stats.total_macs

# # (add-multiplies per forward pass) * (2 FLOPs/add-multiply) * (3 for forward and backward pass) * (number of examples in dataset) 
# training_flops: float  = macs_per_call * 2 * flops_per_call / macs_per_call * (3 * train_step_calles + val_step_calles)
# print(f"FLOPs per call: {flops_per_call:,} = {(flops_per_call * (10 ** -6)):,}M")
# print(f"MACs per call: {macs_per_call:,} = {(macs_per_call * (10 ** -6)):,}M")
del temp_input, temp_target, temp_pred

In [ ]:
# keras.utils.plot_model(
#     model.build_graph(),
#     show_shapes=True,
#     show_dtype=True,
#     show_layer_names=False,
#     expand_nested=True,
#     layer_range=None,
#     show_layer_activations=True
# )

In [ ]:
# model.summary(line_length=125, positions=[0.5, 0.66, 0.83, 1], expand_nested=True)

## Training helper functions

In [ ]:
@tf.function
def contains_pad(inp: tf.Tensor):
    bool_ten = tf.math.equal(inp, pad_ten)
    nonzero_count = tf.math.count_nonzero(bool_ten)
    return nonzero_count > 0

### Train

In [ ]:
# @tf.function(input_signature=(tf.TensorSpec(shape=[batch_size, None], dtype=tf.int32),
#                               tf.TensorSpec(shape=[batch_size, set_size], dtype=tf.int32)))
def train_step(inp: tf.Tensor, outp: tf.Tensor) -> tf.Tensor:
    with tf.GradientTape() as tape:
        pred: tf.Tensor = model([inp, outp], training=True) 
        loss_val: tf.Tensor = loss_func(y_true = outp, y_pred = pred)
    grads: tf.RaggedTensor = tape.gradient(loss_val, model.trainable_weights)
    optimizer.apply_gradients(zip(grads, model.trainable_weights))
    return tf.math.reduce_mean(loss_val), acc(outp, pred)

In [ ]:
# @tf.function
# def test_train_step():
#     exm_inp = tf.random.uniform(shape=[batch_size, 32], dtype=tf.int32, maxval=7000, minval=0)
#     exm_inp2 = tf.random.uniform(shape=[batch_size, 64], dtype=tf.int32, maxval=7000, minval=0)
#     exm_tar = tf.random.uniform(shape=[batch_size, set_size], dtype=tf.int32, maxval=7000, minval=0)
#     train_step(exm_inp, exm_tar)
#     train_step(exm_inp2, exm_tar)
    
# test_train_step()

In [ ]:
@tf.function(input_signature=[tf.TensorSpec(shape=[batch_size, max_seq_len], dtype=tf.int32)])
def train(batch: tf.Tensor) -> tf.TensorSpec(shape=[], dtype=f_type):
    per_gen_loss: tf.Tensor = tf.zeros([num_sets], dtype=f_type)
    per_gen_acc: tf.Tensor = tf.zeros([num_sets], dtype=f_type)
    i = 0
    while i < num_sets:
        # The input is of size set_size-TAKE_TO_ACCOUNT
        already_predicted: int = i * (set_size + 1)
        start_from: int = max(0, already_predicted - max_seq_len)
        inp: tf.Tensor = batch[:, start_from:(i + 1) * set_size]
        have_pad = tf.map_fn(contains_pad, inp, fn_output_signature=tf.bool, parallel_iterations=batch_size)
        if tf.get_static_value(tf.math.reduce_all(have_pad)):
            break
        outp: tf.TensorSpec(shape=[batch_size, set_size]) = batch[:, (i + 1) * set_size:(i + 2) * set_size]
        loss_val, acc_val = train_step(inp, outp)
        one_hot_loss = tf.one_hot([i], num_sets, dtype = f_type) * loss_val
        one_hot_acc = tf.one_hot([i], num_sets, dtype = f_type) * acc_val
        per_gen_loss += one_hot_loss
        per_gen_acc += one_hot_acc
        i += 1
    return tf.math.reduce_mean(per_gen_loss[:i]), tf.math.reduce_mean(per_gen_acc[:i])
    

### Validate

In [ ]:
@tf.function(input_signature=(tf.TensorSpec(shape=[batch_size, None], dtype=tf.int32),
                              tf.TensorSpec(shape=[batch_size, set_size], dtype=tf.int32)))
def val_step(inp: tf.Tensor, outp: tf.Tensor) -> tf.Tensor:
    pred = model([inp, outp], training=False)
    loss_val = loss_func(y_true = outp, y_pred = pred)
    return tf.math.reduce_mean(loss_val), acc(outp, pred)

In [ ]:
@tf.function(input_signature=[tf.TensorSpec(shape=[batch_size, max_seq_len], dtype=tf.int32)])
def validate(batch: tf.Tensor) -> tf.TensorSpec(shape=[], dtype=f_type):
    per_generation_loss: tf.Tensor = tf.zeros([num_sets], dtype=f_type)
    per_gen_acc: tf.Tensor = tf.zeros([num_sets], dtype=f_type)
    i = 0
    while i < num_sets:
        # The input is of size set_size-TAKE_TO_ACCOUNT
        already_predicted: int = i * (set_size + 1)
        start_from: int = max(0, already_predicted - max_seq_len)
        inp: tf.Tensor = batch[:, start_from:(i + 1) * set_size]
        have_pad = tf.map_fn(contains_pad, inp, fn_output_signature=tf.bool, parallel_iterations=batch_size)
        if tf.get_static_value(tf.math.reduce_all(have_pad)):
            break
        loss_val, acc_val = val_step(inp, outp)
        one_hot_loss = tf.one_hot([i], num_sets, dtype=f_type) * loss_val
        one_hot_acc = tf.one_hot([i], num_sets, dtype=f_type) * acc_val
        per_gen_loss += one_hot_loss
        per_gen_acc += one_hot_acc
        i += 1
    return tf.math.reduce_mean(per_gen_loss[:i]), tf.math.reduce_mean(per_gen_acc[:i])

## Callbacks

In [ ]:
best_loss = float("inf")
best_model = None

In [ ]:
def check_point(model: SeTransformer, train_loss, train_acc, val_loss, val_acc, test_loss = None, test_acc = None):
    """Saves the model at the end of each epoch"""
    # (add-multiplies per forward pass) * (2 FLOPs/add-multiply) * 
    # * (3 for forward and backward pass) * (number of examples in dataset)
    global best_model
    global last_save_time
    last_save_time = time.time()
    best_model = model
    num_ops: float  = macs_per_call * 2 * flops_per_call / macs_per_call * (3 * train_step_calles + val_step_calles)
    keras.models.save_model(model = model, filepath = "model.pb", save_format='tf', overwrite=True)
    wandb.log({"model train loss": train_loss, "model train acc": train_acc, "model val loss": val_loss, "model val acc": val_acc, 
              "model test loss": test_loss, "model test acc": test_acc})
    art = wandb.Artifact(f"{wandb.run.id}-best model", type='my_model', description = f"the model after {num_ops:,} operations")
    art.add_file("model.pb")
    run.log_artifact(artifact)
    print("Saved checkpoint")

In [ ]:
def on_val_batch_end(train_loss: float, train_acc: float ,val_loss: float, val_acc: float) -> bool:
    """A callback after every val batch
    returns True if the model should stop training and False else"""
    global best_loss
    global best_model
    global last_save_time
    if time.time() - last_save_time > 1800.0 and val_loss < math.log(vocab_size) and val_loss < best_loss:
        best_loss = val_loss
        # If the last save is more than a half hour (1800 sec) ago
        # and if the predictions are better than randon and 
        check_point(model, train_loss, train_acc, val_loss, val_acc)
        return False
    elif train_loss < 0.01:
        title: str = "Over fitting or data leak"
        message = f"Training loss is {train_loss} and val loss is {val_loss} in the latest batch"
        wandb.alert(title=title, text=message)
        print(title)
        print(message)
        return True
    elif time.time() - last_save_time > 18000.0 and train_loss >= math.log(vocab_size):
        # if the prob of every token is 1/vocab_size, the loss is
        # -ln(1/vocab_size) = ln(vocab_size) 
        # by the logrithem rule log(a^x)=xlog(a) where x = -1
        # if after 5 hours of training, the model predictions are still random
        title: str = "Under fitting"
        message = f"train loss: {train_loss} train acc: {train_acc}, val loss: {val_loss}, val acc: {val_acc} in the latest batch"
        wandb.alert(title=title, text=message)
        print(title)
        print(message)
        return True
    return Flase

## The actual training loop!

In [ ]:
def train_loop():
    epochs: int = 1000000  # Train until the cloud disconnects or the model stops improving
    per_epoch_train_loss: List[float] = []
    per_epoch_val_loss: List[float] = []
    per_epoch_train_acc: List[float] = []
    per_epoch_val_acc: List[float] = []
    print(f"number of train batches per epoch: {len(list_train_set)}")
    last_save_time = time.time()
    for epoch in range(epochs):
        print(f"epoch number: {epoch}")
        per_batch_train_loss: List[float] = []
        per_batch_val_loss: List[float] = []
        per_batch_train_acc: List[float] = []
        per_batch_val_acc: List[float] = []
        for batch_num in tqdm.tqdm(range(len(list_train_set))):  # tqdm is a progress bar
            train_loss, train_acc = train(list_train_set[batch_num])
            float_train_loss = keras.backend.eval(train_loss).item()
            per_batch_train_loss.append(float_train_loss)
            float_train_acc = keras.backend.eval(train_acc).item()
            per_batch_train_acc.append(float_train_acc)
            wandb.log({"epoch": epoch, "batch": batch_num, "batch train loss": float_train_loss, "batch train_acc": float_train_acc})
            if batch_num % 8 == 0:  # 8 = #training batches/#val batches
                # because training set is 80% of the data and val set is 10%
                next_val_batch: tf.Tensor = list_val_set[batch_num // 8]
                val_loss, val_acc = validate(next_val_batch)
                float_val_loss = keras.backend.eval(val_loss).item()
                per_batch_val_loss.append(float_val_loss)
                float_val_acc = keras.backend.eval(val_acc).item()
                per_batch_val_acc.append(float_val_acc)
                wandb.log({"epoch": epoch, "batch": batch_num, "batch val loss": float_val_loss, "batch train_acc": float_val_acc})
                on_val_batch_end(float_train_loss, float_train_acc, float_val_loss, float_val_acc)
        epoch_train_loss = statistics.mean(per_batch_train_loss)
        epoch_train_acc = statistics.mean(per_batch_train_acc)
        epoch_val_loss = statistics.mean(per_batch_val_loss)
        epoch_val_acc = statistics.mean(per_batch_val_acc)
        per_epoch_train_loss.append(epoch_train_loss)
        per_epoch_train_acc.append(epoch_train_acc)
        per_epoch_val_loss.append(epoch_val_loss)
        per_epoch_val_acc.append(epoch_val_acc)
        print(f"train loss: {epoch_train_loss}")
        print(f"train acc: {epoch_train_acc}")
        print(f"val loss: {epoch_val_loss}")
        print(f"val acc: {epoch_val_acc}")
        if len(per_epoch_val_loss) > 1:
            if epoch_val_loss >= per_epoch_val_loss[-2]:
                print("Validation loss increased. Stopped training")
                return epoch_train_loss, epoch_val_loss, epoch_train_acc, epoch_val_acc
        print("Saved checkpoint")

In [ ]:
!export AUTOGRAPH_VERBOSITY=10
train_loss, val_loss, train_acc, val_acc = train_loop()

## After training

In [ ]:
if best_model:
    global model
    model = tf.lite.TFLiteConverter.from_saved_model("model.pb")
    test_loss, test_acc = statistics.mean([validate(test_batch) for test_batch in tqdm.tqdm(list_test_set)])
    print(f"Test loss: {test_loss}")
    print(f"Test acc: {test_acc}")
    wandb.log({"test loss": test_loss, "text acc": test_acc})